### Ingesting Elevation Data to Predicting Fires' Data Sources

In [1]:
import os
import pandas as pd
import requests

### Perform API Calls to USGS

In [2]:
# Base URL for USGS Elevation Point Query Service and coordinates.
base_url = 'https://nationalmap.gov/epqs/pqs.php'

In [3]:
def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. Add the elevation values as a new column."""
    # Define the RESTful query parameters

    params = {
        'x': float(df[lon_column]),
        'y': float(df[lat_column]),
        'units': 'feet',
        'output': 'json'}

    #Makes Request
    result = requests.get(base_url, params=params)
    
    #Parse dictionary for elevation
    elevation = result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

    #Return the elevation float
    return elevation 

### 1) USDA National Fires Database_2003 to 2015 

### Connect to National Fires Database

Data Source: https://www.fs.usda.gov/rds/archive/catalog/RDS-2 013-0009.4  

USDA Data Description: https://www.fs.fed.us/rm/pubs_other/rmrs_2014_short_k001.pdf

In [4]:
firesdb = pd.read_csv('fires_db.csv')

C:\Users\mharr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
firesdb.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,A,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,A,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [6]:
firesdb.columns

Index(['Unnamed: 0', 'OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE',
       'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')

In [7]:
firesdb.shape

(1880465, 40)

In [8]:
firesdb_west = firesdb[(firesdb.LATITUDE > 34) & (firesdb.LONGITUDE < -114)]

In [9]:
firesdb_west.shape

(331982, 40)

In [10]:
df = firesdb_west.head()

In [11]:
df['ELEVATION'] = df.apply(elevation_function, args=('LATITUDE', 'LONGITUDE'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df.head()

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,...,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,ELEVATION
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,...,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,3033.88
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,6008.07
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,3440.45
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,7748.89
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,...,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,7578.20


In [13]:
df[['FIRE_NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].head(1)

,FIRE_NAME,LATITUDE,LONGITUDE,ELEVATION
0,FOUNTAIN,40.036944,-121.005833,3033.88


### 2) USDA Burn Severity Fire Occurence Data_2003 to 2015

### Connect to Burn Severity Databases

Data Source 1: https://data.fs.usda.gov/geodata/edw/datasets.php?xmlKeyword=monitoring+trends+in+burn+severity  

Data Source 2: https://enterprisecontentnew-usfs.hub.arcgis.com/datasets/monitoring-trends-in-burn-severity-fire-occurrence-locations-feature-layer/data


In [14]:
burnsevocc = pd.read_csv('Monitoring_Trends_in_Burn_Severity_Fire_Occurrence_Locations_Feature_Layer.csv')

In [15]:
burnsevocc.columns

Index(['X', 'Y', 'OBJECTID', 'FIRE_ID', 'ASMNT_TYPE', 'PRE_ID', 'POST_ID',
       'NODATA_THRESHOLD', 'LOW_THRESHOLD', 'MODERATE_THRESHOLD',
       'HIGH_THRESHOLD', 'LATITUDE', 'LONGITUDE', 'FIRE_TYPE', 'FIRE_NAME',
       'GREENNESS_THRESHOLD', 'IG_DATE', 'ACRES'],
      dtype='object')

In [16]:
burnsevbound = pd.read_csv('Monitoring_Burned_Area_Boundaries_Feature_Layer.csv')

In [17]:
burnsevbound.columns

Index(['OBJECTID', 'ACRES', 'FIRE_ID', 'FIRE_NAME', 'YEAR', 'STARTMONTH',
       'STARTDAY', 'SHAPE_Length', 'SHAPE_Area', 'FIRE_TYPE'],
      dtype='object')

In [18]:
### Create new dataframe, to then append data you want.

burnsev = pd.merge(burnsevocc, 
                   burnsevbound[['FIRE_ID', 'YEAR', 'STARTMONTH', 'STARTDAY', 'SHAPE_Length', 'SHAPE_Area']],
                     on ='FIRE_ID', 
                    how ='left')
burnsev.head()

,X,Y,OBJECTID,FIRE_ID,ASMNT_TYPE,PRE_ID,POST_ID,NODATA_THRESHOLD,LOW_THRESHOLD,MODERATE_THRESHOLD,...,FIRE_TYPE,FIRE_NAME,GREENNESS_THRESHOLD,IG_DATE,ACRES,YEAR,STARTMONTH,STARTDAY,SHAPE_Length,SHAPE_Area
0,-81.280003,25.952005,1001,FL2595208128019860107,Initial (SS),NaN,50160421986000,-9999,460,-9999,...,Prescribed Fire,BASSLK TRR,-9999,1986-01-07T00:00:00.000Z,2750,1986,1,7,23991.127498,1.382207e+07
1,-97.235007,30.130005,1002,TX3015109722520110904,Extended,50270392010235,70270392012200,-970,70,293,...,Wildfire,BASTROP COUNTY COMPLEX,-150,2011-09-04T00:00:00.000Z,31838,2011,9,4,99452.447798,1.727783e+08
2,-107.527011,43.639007,1003,WY4363910752719960825,Initial,50360301995241,50360301996200,-970,110,286,...,Wildfire,BATES,-150,1996-08-25T00:00:00.000Z,47060,1996,8,25,160333.576454,3.637458e+08
3,-102.297009,31.818005,1004,TX3178810230220110403,Initial (SS),NaN,50300382011100,-9999,-60,-335,...,Wildfire,BATES FIELD FIRE,-9999,2011-04-03T00:00:00.000Z,3186,2011,4,3,24089.508521,1.791484e+07
4,-104.973009,33.180005,1005,NM3322210500120110807,Initial (SS),NaN,50320372011300,-9999,-70,-200,...,Wildfire,BATON ROUGE COMPLEX,-9999,2011-08-07T00:00:00.000Z,21641,2011,8,7,85216.044386,1.253726e+08


In [19]:
### Remove duplicated data columns.

burnsev = burnsev.drop(['X', 'Y'], 1)

In [20]:
burnsev.columns

Index(['OBJECTID', 'FIRE_ID', 'ASMNT_TYPE', 'PRE_ID', 'POST_ID',
       'NODATA_THRESHOLD', 'LOW_THRESHOLD', 'MODERATE_THRESHOLD',
       'HIGH_THRESHOLD', 'LATITUDE', 'LONGITUDE', 'FIRE_TYPE', 'FIRE_NAME',
       'GREENNESS_THRESHOLD', 'IG_DATE', 'ACRES', 'YEAR', 'STARTMONTH',
       'STARTDAY', 'SHAPE_Length', 'SHAPE_Area'],
      dtype='object')

In [21]:
burnsev_west = burnsev[(burnsev.LATITUDE > 34) & (burnsev.LONGITUDE < -114)]

In [22]:
burnsev_west.shape

(6087, 21)

In [23]:
df2 = burnsev_west.head()

In [24]:
df2['ELEVATION'] = df2.apply(elevation_function, args=('LATITUDE', 'LONGITUDE'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df2.head()

,OBJECTID,FIRE_ID,ASMNT_TYPE,PRE_ID,POST_ID,NODATA_THRESHOLD,LOW_THRESHOLD,MODERATE_THRESHOLD,HIGH_THRESHOLD,LATITUDE,...,FIRE_NAME,GREENNESS_THRESHOLD,IG_DATE,ACRES,YEAR,STARTMONTH,STARTDAY,SHAPE_Length,SHAPE_Area,ELEVATION
5,1006,ID4621111458820070810,Extended,50410282006226,70410282008200,-970,110,260,460,46.211,...,BATTLE,-150,2007-08-10T00:00:00.000Z,3363,2007,8,10,34170.203844,2.841447e+07,7154.42
6,1007,NV4055011533719950806,Initial,50400321995205,50400321995200,-970,160,354,600,40.550,...,BATTLE CREEK,-150,1995-08-06T00:00:00.000Z,11424,1995,8,6,64185.230737,8.016170e+07,6538.36
9,1010,OR4541311664220070713,Extended,50420282006201,50420282008200,-970,50,175,345,45.413,...,BATTLE CREEK COMPLEX (BATTLE CREEK),-150,2007-07-13T00:00:00.000Z,79114,2007,7,13,255176.362101,6.496413e+08,4111.35
20,1021,CA3904112055819870831,Extended,50430331987204,50430331988200,-970,100,291,525,39.041,...,BIG,-150,1987-08-31T00:00:00.000Z,1048,1987,8,31,15792.508016,7.041842e+06,5310.78
23,1024,ID4375011641020100728,Initial (SS),NaN,50420302010200,-9999,-50,-9999,-9999,43.750,...,BIG,-9999,2010-07-28T00:00:00.000Z,4748,2010,7,28,33340.390004,3.683527e+07,2722.32


In [26]:
df2[['FIRE_NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']].head(1)

,FIRE_NAME,LATITUDE,LONGITUDE,ELEVATION
5,BATTLE,46.211,-114.588,7154.42


### 3) USDA Burn Emissions Data_2003 to 2015

Data Source: https://www.fs.usda.gov/rds/archive/catalog/RDS-2017-0039


In [27]:
emdata = pd.read_csv('emissions_year2003to2015.csv')

In [28]:
emdata.head()

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,...,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
0,0,2008,359,-81.0384,25.1958,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0
1,1,2008,359,-81.0404,25.1984,4536,3,1600,62500.0,6220.097576,...,157.185824,4.593092,24.292355,0.022757,0.080441,3,1,81,2,0
2,2,2008,359,-81.0380,25.1981,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0
3,3,2008,359,-81.0331,25.1973,4536,95,0,62500.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3,1,81,2,0
4,4,2008,359,-81.0594,25.2035,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0


In [29]:
emdata.columns

Index(['id', 'year', 'doy', 'longitude', 'latitude', 'grid10k', 'covertype',
       'fuelcode', 'area_burned', 'prefire_fuel', 'consumed_fuel', 'ECO2',
       'ECO', 'ECH4', 'EPM2.5', 'cwd_frac', 'duff_frac', 'fuel_moisture_class',
       'burn_source', 'burnday_source', 'BSEV', 'BSEV_flag'],
      dtype='object')

In [30]:
emdata.shape

(7254534, 22)

In [31]:
emdata_west = emdata[(emdata.latitude > 34) & (emdata.longitude < -114)]

In [32]:
emdata_west.shape

(2121764, 22)

In [33]:
df3 = emdata_west.head()

In [34]:
df3['elevation'] = df3.apply(elevation_function, args=('latitude', 'longitude'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
df3.head()

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,...,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag,elevation
150819,150819,2008,286,-118.3726,34.2857,55814,2,2,62500.0,234.371826,...,0.565305,2.151533,0.0,0.0,3,1,15,2,0,1240.99
150820,150820,2008,286,-118.3700,34.2862,55814,21,0,62500.0,0.000000,...,0.000000,0.000000,0.0,0.0,3,1,15,2,0,1203.01
150835,150835,2008,286,-118.3733,34.2879,55814,0,0,62500.0,0.000000,...,0.000000,0.000000,0.0,0.0,3,1,15,2,0,1311.27
150836,150836,2008,286,-118.3706,34.2884,55814,2,2,62500.0,241.657416,...,0.582878,2.218415,0.0,0.0,3,1,15,3,0,1402.38
150837,150837,2008,286,-118.3679,34.2889,55814,2,2,62500.0,1779.028992,...,4.291018,16.331486,0.0,0.0,3,1,15,3,0,1270.10


In [36]:
df3[['year', 'latitude', 'longitude', 'elevation']].head(1)

,year,latitude,longitude,elevation
150819,2008,34.2857,-118.3726,1240.99


### 4) NASA V1 Data

In [37]:
NASAV1 = pd.read_csv('NASA_V1_combined.csv')

C:\Users\mharr\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
NASAV1.head()

,acq_date,acq_time,bright_ti4,bright_ti5,confidence,daynight,frp,instrument,latitude,longitude,satellite,scan,track,type,version
0,2019-10-01,612,307.3,288.6,n,N,2.3,VIIRS,41.46153,-81.67585,N,0.69,0.74,NaN,1.0NRT
1,2019-10-01,612,312.8,288.3,n,N,2.1,VIIRS,41.45969,-81.67487,N,0.69,0.74,NaN,1.0NRT
2,2019-10-01,612,336.8,287.2,n,N,4.8,VIIRS,34.66630,-80.58686,N,0.55,0.68,NaN,1.0NRT
3,2019-10-01,612,307.7,286.4,n,N,1.7,VIIRS,37.23220,-82.03742,N,0.69,0.74,NaN,1.0NRT
4,2019-10-01,612,297.7,285.9,n,N,2.3,VIIRS,37.22986,-82.04183,N,0.69,0.74,NaN,1.0NRT


In [39]:
NASAV1.shape

(10820816, 15)

In [40]:
NASAV1_west = NASAV1[(NASAV1.latitude > 34) & (NASAV1.longitude < -114)]

In [41]:
NASAV1_west.head()

,acq_date,acq_time,bright_ti4,bright_ti5,confidence,daynight,frp,instrument,latitude,longitude,satellite,scan,track,type,version
357,2019-10-01,930,295.5,266.4,n,N,0.7,VIIRS,48.59168,-116.28932,N,0.44,0.38,NaN,1.0NRT
358,2019-10-01,930,296.0,268.9,n,N,0.8,VIIRS,48.59225,-116.30074,N,0.44,0.38,NaN,1.0NRT
359,2019-10-01,930,306.4,269.0,n,N,1.4,VIIRS,48.59498,-116.29317,N,0.44,0.38,NaN,1.0NRT
360,2019-10-01,930,295.0,269.2,n,N,1.2,VIIRS,46.19579,-121.32298,N,0.42,0.46,NaN,1.0NRT
361,2019-10-01,930,304.8,268.4,n,N,1.2,VIIRS,46.20880,-121.32532,N,0.42,0.46,NaN,1.0NRT


In [42]:
NASAV1_west.shape

(3800917, 15)

In [43]:
df4 = NASAV1_west.head()

In [44]:
df4['longitude']

357   -116.28932
358   -116.30074
359   -116.29317
360   -121.32298
361   -121.32532
Name: longitude, dtype: float64

In [45]:
df4['elevation'] = df4.apply(elevation_function, args=('latitude', 'longitude'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df4.head()

,acq_date,acq_time,bright_ti4,bright_ti5,confidence,daynight,frp,instrument,latitude,longitude,satellite,scan,track,type,version,elevation
357,2019-10-01,930,295.5,266.4,n,N,0.7,VIIRS,48.59168,-116.28932,N,0.44,0.38,NaN,1.0NRT,4323.52
358,2019-10-01,930,296.0,268.9,n,N,0.8,VIIRS,48.59225,-116.30074,N,0.44,0.38,NaN,1.0NRT,3808.78
359,2019-10-01,930,306.4,269.0,n,N,1.4,VIIRS,48.59498,-116.29317,N,0.44,0.38,NaN,1.0NRT,4264.83
360,2019-10-01,930,295.0,269.2,n,N,1.2,VIIRS,46.19579,-121.32298,N,0.42,0.46,NaN,1.0NRT,4405.78
361,2019-10-01,930,304.8,268.4,n,N,1.2,VIIRS,46.20880,-121.32532,N,0.42,0.46,NaN,1.0NRT,4719.52


In [47]:
df4[['acq_date', 'latitude', 'longitude', 'elevation']].head(1)

,acq_date,latitude,longitude,elevation
357,2019-10-01,48.59168,-116.28932,4323.52


### 5) NASA M6 Data

In [48]:
NASAM6 = pd.read_csv('fire_archive_M6_110066.csv')

In [49]:
NASAM6.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0
1,19.3739,-155.1130,318.8,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,288.7,98.4,N,2
2,19.3723,-155.1197,316.0,3.1,1.7,2003-01-01,914,Terra,MODIS,90,6.2,288.3,85.0,N,2
3,19.3589,-155.1107,325.2,3.1,1.7,2003-01-01,914,Terra,MODIS,100,6.2,292.0,131.8,N,2
4,19.3573,-155.1174,319.0,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,290.3,97.3,N,2


In [50]:
NASAM6.shape

(2159468, 15)

In [51]:
NASAM6_west = NASAM6[(NASAM6.latitude > 34) & (NASAM6.longitude < -114)]

In [52]:
NASAM6_west.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
78,34.6054,-117.3376,306.5,1.0,1.0,2003-01-02,1846,Terra,MODIS,64,6.2,289.0,6.9,D,0
79,35.2934,-118.4130,301.8,1.0,1.0,2003-01-02,1846,Terra,MODIS,49,6.2,285.5,5.5,D,0
80,35.2529,-118.1982,302.4,1.0,1.0,2003-01-02,1846,Terra,MODIS,52,6.2,283.7,6.2,D,0
178,34.6014,-117.3407,308.7,1.0,1.0,2003-01-03,2102,Aqua,MODIS,65,6.2,294.0,6.9,D,0
179,35.3549,-118.9819,301.8,1.0,1.0,2003-01-03,2102,Aqua,MODIS,43,6.2,289.5,2.9,D,0


In [53]:
NASAM6_west.shape

(724639, 15)

In [54]:
df5 = NASAM6.head()

In [55]:
df5['elevation'] = df5.apply(elevation_function, args=('latitude', 'longitude'), axis=1)

C:\Users\mharr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
df5.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,elevation
0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,710.55
1,19.3739,-155.1130,318.8,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,288.7,98.4,N,2,2354.58
2,19.3723,-155.1197,316.0,3.1,1.7,2003-01-01,914,Terra,MODIS,90,6.2,288.3,85.0,N,2,2407.36
3,19.3589,-155.1107,325.2,3.1,1.7,2003-01-01,914,Terra,MODIS,100,6.2,292.0,131.8,N,2,2260.03
4,19.3573,-155.1174,319.0,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,290.3,97.3,N,2,2311.24


In [57]:
df5[['acq_date', 'latitude', 'longitude', 'elevation']].head(1)

,acq_date,latitude,longitude,elevation
0,2003-01-01,38.8142,-93.5539,710.55
